# Generative Fill example on Amazon SageMaker using DLC container.

In this notebook, we explore how to build generative fill application and host Stable Diffusion 3.0 on SageMaker endpoint using BYOC (Bring-your-own-container).

In this notebook, under the hood we use ComflyUI and extensions to generate image. 

## Build Docker image and push to ECR.

Initialize the variables for SageMaker default bucket, role, and AWS account ID, and current AWS region.

In [11]:
import sagemaker
import boto3 

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role="AmazonSageMaker-ExecutionRole-20220920T203057"

account_id = boto3.client("sts").get_caller_identity().get("Account")
region_name = boto3.session.Session().region_name
inference_image="sd3-compyui-notebook-7-17"

In [ ]:
%%sh -s "$region_name" "$role" "$bucket" "$account_id" "$inference_image"
region=$1
account=$4
inference_image=$5
echo $region   $account
aws ecr get-login-password --region $region | docker login --username AWS --password-stdin $account.dkr.ecr.$region.amazonaws.com.cn

inference_fullname=$account.dkr.ecr.$region.amazonaws.com.cn/$inference_image:latest
echo $inference_fullname

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${inference_image}" --region $region || aws ecr create-repository --repository-name "$inference_image" --region $region

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${inference_image}" --region ${region}
    echo "I am here created new ECR Repo $inference_image"
fi

AWSchinaAccount="727897471807"

# Get the login command from ECR and execute it directly
docker login -u AWS -p $(aws ecr get-login-password --region $region) $AWSchinaAccount.dkr.ecr.$region.amazonaws.com.cn

aws ecr set-repository-policy \
    --repository-name "${inference_image}" \
    --policy-text "file://ecr-policy.json" \
    --region ${region}

#docker build -t ${inference_image} -f Dockerfile.inference . --build-arg REGION=$region
docker build --no-cache -t ${inference_image} -f Dockerfile.inference . --build-arg REGION=$region

docker tag ${inference_image} ${inference_fullname}

docker push ${inference_fullname}

Upload the dummy file to S3 to meet the requirement of SageMaker Endpoint for model data.

In [ ]:
model_data = "s3://{0}/SD3-ComfyUI-fake/data/model.tar.gz".format(bucket)
!touch dummy
!tar czvf model.tar.gz dummy
!rm dummy
!aws s3 cp model.tar.gz $model_data

## Deploy to SageMaker Asychronous Endpoint

Initialized the variables for URI of Docker Inference Endpoint.

In [ ]:
model_name = None
image_uri = "{0}.dkr.ecr.{1}.amazonaws.com.cn/{2}:latest".format(
    account_id, region_name, inference_image
)
print(image_uri)

Define the dummy models configuration for models.

In [15]:
import json

huggingface_models = [
    {
    }
]

model_environment = {
}

Define the model, instance type and instance initial count for SageMaker endpoint.

In [16]:
from sagemaker.model import Model
from sagemaker.predictor import Predictor

model = Model(
    name=model_name,
    model_data=model_data,
    role=role,
    image_uri=image_uri,
    env=model_environment,
    predictor_cls=Predictor,
)

instance_type = "ml.g5.xlarge"
instance_count = 1

Define the SageMaker Asychronous Inference config

In [18]:
from sagemaker.async_inference import AsyncInferenceConfig

async_config = AsyncInferenceConfig(
    output_path="s3://{0}/{1}/asyncinvoke/out/".format(bucket, "sd3-comfyui-7-18"),
    max_concurrent_invocations_per_instance=4,
    # Optionally specify Amazon SNS topics
    # notification_config = {
    # "SuccessTopic": "arn:aws:sns:<aws-region>:<account-id>:<topic-name>",
    # "ErrorTopic": "arn:aws:sns:<aws-region>:<account-id>:<topic-name>",
    # }
)

Here we use asynchronous inference since asynchronous inference is more suitable for workloads with large payload sizes and long inference processing times. 

In [ ]:
predictor = model.deploy(
    instance_type=instance_type,
    initial_instance_count=instance_count,
    container_startup_health_check_timeout=600,
    async_inference_config=async_config,
)

## Generate initial image using text prompt

In [21]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

predictor.serializer = JSONSerializer()
predictor.deserializer = JSONDeserializer()

inputs = {
    "prompt": {
        "3": {
            "inputs": {
                "seed": 96228230156886,
                "steps": 20,
                "cfg": 8,
                "sampler_name": "euler",
                "scheduler": "normal",
                "denoise": 1,
                "model": [
                    "4",
                    0
                ],
                "positive": [
                    "6",
                    0
                ],
                "negative": [
                    "7",
                    0
                ],
                "latent_image": [
                    "5",
                    0
                ]
            },
            "class_type": "KSampler",
            "_meta": {
                "title": "KSampler"
            }
        },
        "4": {
            "inputs": {
                "ckpt_name": "sd3_medium_incl_clips.safetensors"
            },
            "class_type": "CheckpointLoaderSimple",
            "_meta": {
                "title": "Load Checkpoint"
            }
        },
        "5": {
            "inputs": {
                "width": 512,
                "height": 512,
                "batch_size": 1
            },
            "class_type": "EmptyLatentImage",
            "_meta": {
                "title": "Empty Latent Image"
            }
        },
        "6": {
            "inputs": {
                "text": "beautiful scenery nature glass bottle landscape, , purple galaxy desk",
                "clip": [
                    "4",
                    1
                ]
            },
            "class_type": "CLIPTextEncode",
            "_meta": {
                "title": "CLIP Text Encode (Prompt)"
            }
        },
        "7": {
            "inputs": {
                "text": "text, watermark",
                "clip": [
                    "4",
                    1
                ]
            },
            "class_type": "CLIPTextEncode",
            "_meta": {
                "title": "CLIP Text Encode (Prompt)"
            }
        },
        "8": {
            "inputs": {
                "samples": [
                    "3",
                    0
                ],
                "vae": [
                    "4",
                    2
                ]
            },
            "class_type": "VAEDecode",
            "_meta": {
                "title": "VAE Decode"
            }
        },
        "9": {
            "inputs": {
                "filename_prefix": "ComfyUI",
                "images": [
                    "8",
                    0
                ]
            },
            "class_type": "SaveImage",
            "_meta": {
                "title": "Save Image"
            }
        }
    }
}

prediction = predictor.predict(inputs)

Helper function for S3.

In [6]:
import json
import io

s3_resource = boto3.resource("s3")

def get_bucket_and_key(s3uri):
    pos = s3uri.find("/", 5)
    bucket = s3uri[5:pos]
    key = s3uri[pos + 1 :]
    return bucket, key



Wait until the asychronous inference is done in case we use asynchronous inference for image generation. 

In [7]:
from sagemaker.async_inference.waiter_config import WaiterConfig

print(f"Response object: {prediction}")
#print(f"Response output path: {prediction.output_path}")
print("Start Polling to get response:")

import time

start = time.time()

config = WaiterConfig(
    max_attempts=100, delay=10  #  number of attempts  #  time in seconds to wait between attempts
)

prediction.get_result(config)

print(f"Time taken: {time.time() - start}s")

NameError: name 'prediction' is not defined

Process the generated images from asynchronous inference result.

In [22]:
import traceback
from PIL import Image
import uuid
from io import BytesIO
from datetime import datetime
import base64

try:
    output_bucket, output_key = get_bucket_and_key(prediction.output_path)
    output_obj = s3_resource.Object(output_bucket, output_key)
    body = output_obj.get()["Body"].read().decode("utf-8")
    image_object = json.loads(body)["images"][0]
    image = Image.open(BytesIO(base64.b64decode(image_object)))
    image.show()
    initial_image_filename = datetime.now().strftime(f"%Y%m%d%H%M%S-{uuid.uuid4()}.png")
    image.save(initial_image_filename)
except Exception as e:
    traceback.print_exc()
    print(e)

'bytes' object has no attribute 'output_path'


Traceback (most recent call last):
  File "/tmp/ipykernel_6000/758733565.py", line 9, in <module>
    output_bucket, output_key = get_bucket_and_key(prediction.output_path)
AttributeError: 'bytes' object has no attribute 'output_path'


## Resource cleanup.

In [ ]:
predictor.delete_endpoint()